### 经典又兼具备趣味性的Kaggle案例[泰坦尼克号问题](https://www.kaggle.com/c/titanic)
大家都熟悉的『Jack and Rose』的故事，豪华游艇倒了，大家都惊恐逃生，可是救生艇的数量有限，无法人人都有，副船长发话了『lady and kid first！』，所以是否获救其实并非随机，而是基于一些背景有rank先后的。<br>
训练和测试数据是一些乘客的个人信息以及存活状况，要尝试根据它生成合适的模型并预测其他人的存活状况。<br>
对，这是一个二分类问题，很多分类算法都可以解决。

# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [160]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [161]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine = [train_df, test_df]

In [162]:
# C港口登录乘客 女性与男性 fare分布

In [163]:
print("Before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)
combine = [train_df, test_df]

"After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape

('Before', (891, 12), (418, 11), (891, 12), (418, 11))


('After', (891, 11), (418, 10), (891, 11), (418, 10))

In [164]:
train_df['IsGroup'] = 0
tmp = train_df[['Ticket','IsGroup']].groupby('Ticket').count()
isgroup = list(tmp[tmp['IsGroup']>1].index)
isgroup

['110152',
 '110413',
 '110465',
 '111361',
 '113505',
 '113572',
 '113760',
 '113776',
 '113781',
 '113789',
 '113798',
 '113803',
 '113806',
 '11668',
 '11751',
 '11767',
 '11967',
 '12749',
 '13502',
 '13507',
 '1601',
 '16966',
 '17421',
 '17453',
 '17474',
 '19877',
 '19928',
 '19943',
 '19950',
 '19996',
 '220845',
 '230080',
 '230136',
 '230433',
 '231919',
 '237736',
 '239853',
 '239865',
 '24160',
 '243847',
 '244252',
 '244367',
 '248727',
 '248738',
 '250644',
 '250647',
 '250649',
 '250655',
 '2627',
 '26360',
 '2651',
 '2653',
 '2659',
 '2661',
 '2665',
 '2666',
 '2668',
 '2678',
 '2691',
 '2699',
 '28403',
 '2908',
 '29106',
 '29750',
 '3101278',
 '3101295',
 '31027',
 '345764',
 '345773',
 '347054',
 '347077',
 '347080',
 '347082',
 '347088',
 '347742',
 '349237',
 '349909',
 '35273',
 '35281',
 '358585',
 '363291',
 '364516',
 '364849',
 '367226',
 '367230',
 '36928',
 '36947',
 '36973',
 '370129',
 '370365',
 '371110',
 '376564',
 '382652',
 '392096',
 '4133',
 '54636'

In [165]:
# 终于搞定了， 筛选出Ticket 字段中非unique值，将这部分值的IsGroup字段设置为1
train_df.loc[train_df['Ticket'].isin(isgroup), 'IsGroup'] = 1
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,IsGroup
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,0


In [166]:
for dataset in combine:
    dataset['IsGroup'] = 0
    tmp = dataset[['Ticket','IsGroup']].groupby('Ticket').count()
    isgroup = list(tmp[tmp['IsGroup']>1].index)
    dataset.loc[dataset['Ticket'].isin(isgroup), 'IsGroup'] = 1

In [167]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

In [168]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


In [169]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

In [170]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

((891, 11), (418, 11))

In [171]:
# 团体票与否似乎对成活率没有太大影响

In [172]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

In [173]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,IsGroup,Title
0,0,3,0,22,1,0,A/5 21171,7.2500,S,0,1
1,1,1,1,38,1,0,PC 17599,71.2833,C,0,3
2,1,3,1,26,0,0,STON/O2. 3101282,7.9250,S,0,2
3,1,1,1,35,1,0,113803,53.1000,S,1,3
4,0,3,0,35,0,0,373450,8.0500,S,0,1


In [174]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(-0.08, 16]",0.550000
1,"(16, 32]",0.337374
2,"(32, 48]",0.412037
3,"(48, 64]",0.434783
4,"(64, 80]",0.090909


In [175]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,IsGroup,Title,AgeBand
0,0,3,0,1,1,0,A/5 21171,7.2500,S,0,1,"(16, 32]"
1,1,1,1,2,1,0,PC 17599,71.2833,C,0,3,"(32, 48]"
2,1,3,1,1,0,0,STON/O2. 3101282,7.9250,S,0,2,"(16, 32]"
3,1,1,1,2,1,0,113803,53.1000,S,1,3,"(32, 48]"
4,0,3,0,2,0,0,373450,8.0500,S,0,1,"(32, 48]"


In [176]:
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,IsGroup,Title
0,0,3,0,1,1,0,A/5 21171,7.2500,S,0,1
1,1,1,1,2,1,0,PC 17599,71.2833,C,0,3
2,1,3,1,1,0,0,STON/O2. 3101282,7.9250,S,0,2
3,1,1,1,2,1,0,113803,53.1000,S,1,3
4,0,3,0,2,0,0,373450,8.0500,S,0,1


In [177]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,FamilySize,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.136364
7,8,0.000000
8,11,0.000000


In [178]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

,IsAlone,Survived
0,0,0.505650
1,1,0.303538


In [179]:
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

train_df.head()

,Survived,Pclass,Sex,Age,Ticket,Fare,Embarked,IsGroup,Title,IsAlone
0,0,3,0,1,A/5 21171,7.2500,S,0,1,0
1,1,1,1,2,PC 17599,71.2833,C,0,3,0
2,1,3,1,1,STON/O2. 3101282,7.9250,S,0,2,1
3,1,1,1,2,113803,53.1000,S,1,3,0
4,0,3,0,2,373450,8.0500,S,0,1,1


In [180]:
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

'S'

In [181]:
# female 成活率明显高于male

In [182]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.339009


In [183]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,Ticket,Fare,Embarked,IsGroup,Title,IsAlone
0,0,3,0,1,A/5 21171,7.2500,0,0,1,0
1,1,1,1,2,PC 17599,71.2833,1,0,3,0
2,1,3,1,1,STON/O2. 3101282,7.9250,0,0,2,1
3,1,1,1,2,113803,53.1000,0,1,3,0
4,0,3,0,2,373450,8.0500,0,0,1,1


In [184]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Embarked,IsGroup,Title,IsAlone
0,892,3,0,2,330911,7.8292,2,0,1,1
1,893,3,1,2,363272,7.0000,0,0,3,0
2,894,2,0,3,240276,9.6875,2,0,1,1
3,895,3,0,1,315154,8.6625,0,0,1,1
4,896,3,1,1,3101298,12.2875,0,0,3,0


In [185]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"[0, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31]",0.454955
3,"(31, 512.329]",0.581081


In [186]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
    
train_df.head(10)

,Survived,Pclass,Sex,Age,Ticket,Fare,Embarked,IsGroup,Title,IsAlone
0,0,3,0,1,A/5 21171,0,0,0,1,0
1,1,1,1,2,PC 17599,3,1,0,3,0
2,1,3,1,1,STON/O2. 3101282,1,0,0,2,1
3,1,1,1,2,113803,3,0,1,3,0
4,0,3,0,2,373450,1,0,0,1,1
5,0,3,0,1,330877,1,2,0,1,1
6,0,1,0,3,17463,3,0,0,1,1
7,0,3,0,0,349909,2,0,1,4,0
8,1,3,1,1,347742,1,0,1,3,0
9,1,2,1,0,237736,2,1,1,3,0


In [187]:
train_df = train_df.drop("Ticket", axis=1)
train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,IsGroup,Title,IsAlone
0,0,3,0,1,0,0,0,1,0
1,1,1,1,2,3,1,0,3,0
2,1,3,1,1,1,0,0,2,1
3,1,1,1,2,3,0,1,3,0
4,0,3,0,2,1,0,0,1,1


In [188]:
test_df = test_df.drop("Ticket", axis=1)
test_df.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,IsGroup,Title,IsAlone
0,892,3,0,2,0,2,0,1,1
1,893,3,1,2,0,0,0,3,0
2,894,2,0,3,1,2,0,1,1
3,895,3,0,1,1,0,0,1,1
4,896,3,1,1,1,0,0,3,0


In [189]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 8), (891,), (418, 8))

In [203]:
X_test.head()

,Pclass,Sex,Age,Fare,Embarked,IsGroup,Title,IsAlone
0,3,0,2,0,2,0,1,1
1,3,1,2,0,0,0,3,0
2,2,0,3,1,2,0,1,1
3,3,0,1,1,0,0,1,1
4,3,1,1,1,0,0,3,0


In [202]:
X_train.head()

,Pclass,Sex,Age,Fare,Embarked,IsGroup,Title,IsAlone
0,3,0,1,0,0,0,1,0
1,1,1,2,3,1,0,3,0
2,3,1,1,1,0,0,2,1
3,1,1,2,3,0,1,3,0
4,3,0,2,1,0,0,1,1


In [190]:
## Model, predict and solve

Now we are ready to train a model and predict the required solution. There are 60+ predictive modelling algorithms to choose from. We must understand the type of problem and solution requirement to narrow down to a select few models which we can evaluate. Our problem is a classification and regression problem. We want to identify relationship between output (Survived or not) with other variables or features (Gender, Age, Port...). We are also perfoming a category of machine learning which is called supervised learning as we are training our model with a given dataset. With these two criteria - Supervised Learning plus Classification and Regression, we can narrow down our choice of models to a few. These include:

- Logistic Regression
- KNN or k-Nearest Neighbors
- Support Vector Machines
- Naive Bayes classifier
- Decision Tree
- Random Forrest
- Perceptron
- Artificial neural network
- RVM or Relevance Vector Machine

SyntaxError: invalid syntax (<ipython-input-190-f8c51783b2fc>, line 3)

In [191]:
# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

78.56

In [192]:
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
1,Sex,2.144334
6,Title,0.411079
4,Embarked,0.311009
5,IsGroup,0.154774
7,IsAlone,0.116443
2,Age,-0.031806
3,Fare,-0.067176
0,Pclass,-1.008707


In [193]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

83.16

In [194]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

85.3

In [195]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

76.09

In [196]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

77.78

In [197]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

78.45

In [198]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

80.02

In [204]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

87.21

In [205]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('submission.csv', index=False)

In [200]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

87.21

In [201]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,87.21
8,Decision Tree,87.21
1,KNN,85.30
0,Support Vector Machines,83.16
6,Stochastic Gradient Decent,80.02
2,Logistic Regression,78.56
7,Linear SVC,78.45
5,Perceptron,77.78
4,Naive Bayes,76.09


In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
# submission.to_csv('../output/submission.csv', index=False)